# Assigment 2
## Complete your own implementation of a web spider. Combine it with the linguistic analysis code we saw previously (either NLP or distance matrix calculation) and use it to generate summaries of the pages. You can do this in one of two ways, either:

### - Scrape a single page and then run your code on this page.
### - Scrape a series of interlinked pages (e.g. looking for the a href element) and summarise each individual page plus a broad summary of all pages.

[github repo guide]('https://medium.com/analytics-vidhya/simple-text-summarization-using-nltk-eedc36ebaaf8#:~:text=Text%20summarization%20is%20the%20process,relevant%20information%20within%20the%20Text.')

[github repo guide 2]('https://github.com/akashp1712/nlp-akash/blob/master/text-summarization/Word_Frequency_Summarization.py')

**potential issue with algorithm is that long sentences will have advantage over short  
solve this by dividing every sentence score by number of words in the sentence**

** Note that here sentence[:10] is the first 10 character of any sentence, this is to save memory while saving keys of the dictionary**

In [161]:
from bs4 import BeautifulSoup
import requests
import re
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from heapq import nlargest
nltk.download('punkt')
nltk.download('stopwords')


url_1 = 'https://en.wikipedia.org/wiki/Don_Lanphere'

url_2 = 'https://en.wikipedia.org/wiki/Double-Cross_System'

url_3 = 'https://en.wikipedia.org/wiki/Long_Ditton'

## issue of some late paragraphs having random social media links or email or author info
## filter out if p shorter than say 7 words?



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hugho\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hugho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [211]:
def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    article_soup = soup.find_all('p')
    return article_soup

def get_article_content(article_soup):
    content = []
    for item in article_soup:
        content.append(item.get_text())
    #remove shorter sentences to keep headers and non article related paragraphs out of the analysis
    no_short_ps = []
    for sentence in content:
        if len(sentence) > 50:
            no_short_pgs.append(sentence)
        # else:
        #     print('REMOVED SENTENCE:', sentence)
    article_string = ' '.join(no_short_ps)
    return article_string

# get_article_content('https://en.wikipedia.org/wiki/Operation_Hardboiled')

In [115]:
def get_word_freq(content):
    ps = PorterStemmer()
    stop_words = nltk.corpus.stopwords.words('english')
    #remove non-ascii and digihts
    content=re.sub("(\\W|\\d)"," ",content)
    #remove whitespace
    content = content.strip()
    
    tokens = word_tokenize(content)
    word_frequencies = {}
    for tok in tokens:
        tok = ps.stem(tok)
        if tok in stop_words:
            continue
        if tok not in word_frequencies.keys():
            word_frequencies[tok] = 1
        else:
            word_frequencies[tok] += 1
    return word_frequencies

# max_word_freq = max(word_freq.values())

In [116]:

def get_sentence_scores(sentences, freq_table) -> dict:
    sentence_value = dict()

    for sentence in sentences:
        #get sentence word count
        word_count_in_sentence = (len(word_tokenize(sentence)))
        #set initial word count for sentence minus stop words
        word_count_in_sentence_except_stop_words = 0
        #create sentence value table
        for word_val in freq_table:
            if word_val in sentence.lower():
                word_count_in_sentence_except_stop_words += 1
                if sentence in sentence_value:
                    sentence_value[sentence] += freq_table[word_val]
                else:
                    sentence_value[sentence] = freq_table[word_val]

        if sentence in sentence_value:
            sentence_value[sentence] = sentence_value[sentence] / word_count_in_sentence_except_stop_words
    
    return sentence_value

In [232]:
def get_final_summary(sentence_scores, theshold, combined):
    select_length = int(len(sentence_scores)*theshold)
    if select_length > 10:
        select_length = 10
    if combined == True:
        select_length = 15
    summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
    
    final_summary = ' '.join(summary)
    
    return final_summary   

In [234]:
def get_article_summary_from_url(url, theshold):
    article_soup = get_soup(url)

    content_text = get_article_content(article_soup)

    word_freq = get_word_freq(content_text)

    sentence_tokens = sent_tokenize(content_text)
    
    sentence_scores = get_sentence_scores(sentence_tokens, word_freq)
    
    summary = get_final_summary(sentence_scores, theshold, False)
    return summary

# get_article_summary_from_url(url_1, 0.25)



In [184]:
def get_article_links(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    article_body = soup.find_all('p')
    article_links = []
    for item in article_body:
        children = item.findChildren('a')
        for child in children:
            article_links.append(child['href'])
    article_links = article_links[:3]
    new_links = []
    for link in article_links:
        new_links.append('https://en.wikipedia.org' + link)
    new_links.append(url)
    return new_links

In [235]:
def get_linked_article_summaries(url):
    link_list = get_article_links(url)
    summaries = []
    for article_url in link_list:
        summary = get_article_summary_from_url(article_url, 0.2)
        summaries.append(summary)
        print(f'\n SUMMARY for {article_url}: \n{ summary } \n\n\n')
    combined_summary = get_multiple_article_summary(link_list, 0.4)
    print(f'\n COMBINED ARTICLE SUMMARY: \n{ combined_summary } \n\n\n')
    

def get_multiple_article_summary(url_list, threshold):
    big_ole_soup = []
        
    for url in url_list:
        url_soup = get_soup(url)
        contents = get_article_content(url_soup)
        big_ole_soup.append(contents)
    
    content_text = ' '.join(big_ole_soup)

    word_freq = get_word_freq(content_text)
    
    sentence_tokens = sent_tokenize(content_text)
    
    sentence_scores = get_sentence_scores(sentence_tokens, word_freq)
    
    summary = get_final_summary(sentence_scores, threshold, True)
    
    return summary


get_linked_article_summaries(url_2)

#issue where longer articles have heavier weightinig and they dominate the results
# would have to adjust this to weight the articles based on size somehow similar to weighting sentence by word count


 SUMMARY for https://en.wikipedia.org/wiki/World_War_II: 
[294]
 Germany lost a quarter of its pre-war (1937) territory. They agreed on the occupation of post-war Germany, and on when the Soviet Union would join the war against Japan. [63] The first German attack of the war came against the Polish defenses at Westerplatte. In 1944, the Western Allies invaded German-occupied France, while the Soviet Union regained its territorial losses and turned towards Germany and its allies. The Americans favoured a straightforward, large-scale attack on Germany through France. [67]
 On 8 September, German troops reached the suburbs of Warsaw. A peace treaty between Japan and the Allies was signed in 1951. On 4 July 1943, Germany attacked Soviet forces around the Kursk Bulge. Both Germany and the Soviet Union used this proxy war as an opportunity to test in combat their most advanced weapons and tactics. [94]
 On 10 June, Italy invaded France, declaring war on both France and the United Kingdom. 

